In [159]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras_preprocessing.image import ImageDataGenerator
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))


In [41]:
data_label = pd.read_csv('/kaggle/input/fiu-cap5610-spring-2023/train.csv')
test_label = pd.read_csv('/kaggle/input/fiu-cap5610-spring-2023/test.csv')

In [83]:
test_label

,id
0,3991
1,1337
2,83
3,2688
4,4363
...,...
995,947
996,294
997,2068
998,607


In [3]:
data_label.loc[data_label.id == 1]

,id,class
704,1,A0


In [4]:
data_label.describe()

,id
count,4000.000000
mean,2483.224250
std,1445.698791
min,0.000000
25%,1226.750000
50%,2478.000000
75%,3730.250000
max,4999.000000


In [5]:
image_location = '/kaggle/input/fiu-cap5610-spring-2023/images'

In [6]:
feature_name = []
for i in range(0,4096):
    feature_name.append('f' + str(i))
feature_name.append('image_number')

In [7]:
df_image_dataset = pd.DataFrame()#columns=feature_name)

In [8]:
df_image_dataset

""


In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


size = 64

for img_num in range(0, 5000):
    #print(img_num)
    my_img = cv2.imread('/kaggle/input/fiu-cap5610-spring-2023/images/' + str(img_num) + '.png') 
    gray_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

    dim = (size, size)

    # resize image
    resized = cv2.resize(blackAndWhiteImage, dim, interpolation = cv2.INTER_AREA)
    flattened = resized.flatten()


    # Visualize the result
    #plt.imshow(resized)
    #plt.show()

    #print(resized.shape)
    #print(flattened.shape)

    flattened = np.append(flattened, img_num)

    q = pd.DataFrame(flattened)
    q= q.T

    #df_image_dataset = df_image_dataset.T
    df_image_dataset = df_image_dataset.append(q)

#df1.append(df2, ignore_index = True)

In [40]:
df_image_dataset

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,0
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,1
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,2
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,3
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4995
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4996
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4997
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4998


In [14]:
data_label.id

0       1276
1       1333
2       3011
3       3950
4        685
        ... 
3995     757
3996    1444
3997    3977
3998     532
3999     131
Name: id, Length: 4000, dtype: int64

In [42]:
test_label

,id
0,3991
1,1337
2,83
3,2688
4,4363
...,...
995,947
996,294
997,2068
998,607


In [43]:
new = df_image_dataset[4096].isin(data_label.id)
new2 = df_image_dataset[4096].isin(test_label.id)

In [130]:
df_image_dataset_train = df_image_dataset[new]
df_image_dataset_test = df_image_dataset[new2]

In [131]:
df_image_dataset_train = df_image_dataset_train.rename(columns= {4096:'id'})
df_image_dataset_test = df_image_dataset_test.rename(columns= {4096:'id'})

In [132]:
df_image_dataset_test

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,id
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,7
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,8
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,11
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,15
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4976
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4978
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4980
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4985


In [133]:
df_image_dataset_train = pd.merge(df_image_dataset_train, data_label, on='id')

In [134]:
df_image_dataset_train

,0,1,2,3,4,5,6,7,8,9,...,4088,4089,4090,4091,4092,4093,4094,4095,id,class
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,0,D3
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,1,A0
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,2,D0
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,3,C2
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,4,D1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,4995,B0
3996,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,4996,B0
3997,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,4997,A4
3998,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,4998,C1


In [21]:
df_image_dataset_train = df_image_dataset_train.drop(columns=['id'])

In [125]:
df_image_dataset_train

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,id
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,0
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,1
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,2
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,3
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4995
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4996
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4997
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,4998


In [144]:
class_category = pd.get_dummies(df_image_dataset_train['class'])
pd.concat([df_image_dataset_train, class_category], axis=1)

,0,1,2,3,4,5,6,7,8,9,...,C0,C1,C2,C3,C4,D0,D1,D2,D3,D4
0,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,1,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,1,0,0,0,0
3,255,255,255,255,255,255,255,255,255,255,...,0,0,1,0,0,0,0,0,0,0
4,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3996,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3997,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3998,255,255,255,255,255,255,255,255,255,255,...,0,1,0,0,0,0,0,0,0,0


In [145]:
class_category 

,A0,A1,A2,A3,A4,B0,B1,B2,B3,B4,C0,C1,C2,C3,C4,D0,D1,D2,D3,D4
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3996,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3997,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3998,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [76]:
#class_category

In [124]:
train_data = df_image_dataset_train.drop(columns = ['class'])

KeyError: "['class'] not found in axis"

In [62]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3996,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3997,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3998,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [29]:
clf = svm.SVC(gamma=0.001, kernel = 'poly',degree= 7 )

X_train, X_test, y_train, y_test = train_test_split(
    train_data, df_image_dataset_train['class'], test_size=0.00001, shuffle=True
)

In [37]:
y_test

2176    A0
Name: class, dtype: object

In [30]:
clf.fit(X_train, y_train)

# Predict the value of the digit on the test subset
#predicted = clf.predict(X_test)

SVC(degree=7, gamma=0.001, kernel='poly')

In [68]:
X_train.iloc[2897]

0       255
1       255
2       255
3       255
4       255
       ... 
4091    255
4092    255
4093    255
4094    255
4095    255
Name: 2484, Length: 4096, dtype: int64

In [32]:
predicted = clf.predict(X_test)

In [50]:
df_image_dataset_test=df_image_dataset_test.reset_index(drop = True)

In [78]:
one_test = df_image_dataset_test.iloc[0][:4096].values.reshape(1,-1)

In [110]:
test_id = []
test_result = []

for i in range (0, len(df_image_dataset_test)):
    one_test = df_image_dataset_test.iloc[i][:4096].values.reshape(1,-1)
    test_result.append(str(clf.predict(one_test))[2:4])
    test_id.append(df_image_dataset_test.loc[i]['id'])

In [111]:
print(test_id[0], test_result[0])

7 B0


In [112]:
df_test_result = pd.DataFrame(list(zip(test_id, test_result)),
               columns =['id', 'class'])

In [113]:
df_test_result

,id,class
0,7,B0
1,8,D0
2,11,A4
3,15,C1
4,34,C4
...,...,...
995,4976,D4
996,4978,B0
997,4980,A4
998,4985,C3


In [114]:
test_label

,id
0,3991
1,1337
2,83
3,2688
4,4363
...,...
995,947
996,294
997,2068
998,607


In [117]:
result = pd.merge(test_label, df_test_result, on="id")

In [119]:
result.to_csv('submission1_SVM_Ashfaq.csv')

In [146]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3996,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3997,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3998,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [151]:
#MLP
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(300,200,100,50),
                        #max_iter = 300,
                        activation = 'relu',
                        solver = 'adam')

X_train, X_test, y_train, y_test = train_test_split(
    train_data, df_image_dataset_train['class'], test_size=0.2, shuffle=True)


mlp_clf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(hidden_layer_sizes=100)

In [152]:
y_test

,A0,A1,A2,A3,A4,B0,B1,B2,B3,B4,C0,C1,C2,C3,C4,D0,D1,D2,D3,D4
1324,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3043,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2957,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3565,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2285,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2923,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [154]:
x = mlp_clf.predict(X_test)

In [158]:
x[4]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [33]:
accuracy_score(y_test, predicted)

1.0

In [34]:
print(predicted[0])
print(y_test[0])

A0


KeyError: 0

In [114]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [116]:
X_train, X_test, y_train, y_test = train_test_split(
    train_data, class_category, test_size=0.2, shuffle=True
)
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [117]:
predicted = clf.predict(X_test)
print(accuracy_score(y_test, predicted))

0.17625
